In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
import lightgbm as lgbm
import xgboost as xgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier, VotingClassifier
import optuna
import tqdm
import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
sns.countplot(x='target', data=train_df)
ax.set_title('Target Distribution')

In [ ]:
skewValue = train_df.skew(axis=0)

In [ ]:
for item in skewValue:
    print(item)

skewness > 0 : more weight in the left tail of the distribution. skewness < 0 : more weight in the right tail of the distribution.

In [ ]:
train_df.drop(columns=['id']).describe().T.style.bar(subset=['mean'], color='#606ff2')\
                            .background_gradient(subset=['std'], cmap='BrBG')\
                            .background_gradient(subset=['min'], cmap='BrBG')\
                            .background_gradient(subset=['50%'], cmap='BrBG')


Data Transformation: 

In [ ]:
le = LabelEncoder()
train_df['target'] = le.fit_transform(train_df['target'])
train_df.columns


cols = list(train_df.columns)
cols.remove("target")
cols.remove("id")

In [ ]:
train_df.columns

Scaling

In [ ]:
scaler = StandardScaler()
train_df[cols] = scaler.fit_transform(train_df[cols])
test_df[cols] = scaler.transform(test_df[cols])

In [ ]:
X=train_df.drop(['target','id'],axis=1)
Y=train_df['target']

Applying Optuna:

In [ ]:
def objective(trial,data=X,target=Y):
    
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.25,random_state=50)
    params = {'iterations':trial.suggest_int("iterations", 4000, 16000),
              'od_wait':trial.suggest_int('od_wait', 500, 3200),
             'loss_function':'MultiClass',
              'task_type':"GPU",
              'eval_metric':'MultiClass',
              'leaf_estimation_method':'Newton',
              'bootstrap_type': 'Bernoulli',
              'learning_rate' : trial.suggest_uniform('learning_rate',0.01,0.5),
              'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
              'subsample': trial.suggest_uniform('subsample',0,1),
              'random_strength': trial.suggest_uniform('random_strength',10,30),
              'depth': trial.suggest_int('depth',1,6),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
              'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,8),
               }
    model = CatBoostClassifier(**params)  
    model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=100,verbose=False)
        
    y_preds = model.predict_proba(X_test)


    log_loss_multi = log_loss(y_test, y_preds)
    
    return log_loss_multi


In [ ]:
OPTUNA_OPTIMIZATION = True

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=25)
print('Number of finished trials:', len(study.trials))
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

In [ ]:
if OPTUNA_OPTIMIZATION:
    display(optuna.visualization.plot_optimization_history(study))
    display(optuna.visualization.plot_slice(study))

Performing Stratified KFold Cross Validation:

In [ ]:
cat_params = study.best_trial.params
cat_params['loss_function'] = 'MultiClass'
cat_params['eval_metric'] = 'MultiClass'
cat_params['bootstrap_type']= 'Bernoulli'
cat_params['leaf_estimation_method'] = 'Newton'
cat_params['random_state'] = 42
cat_params['task_type']='GPU'
test_preds=None

kf = StratifiedKFold(n_splits = 10 , shuffle = True , random_state = 50)
for fold, (tr_index , val_index) in enumerate(kf.split(X.values , Y.values)):
    
    print("-" * 50)
    print(f"Fold {fold + 1}")
    
    x_train,x_val = X.values[tr_index] , X.values[val_index]
    y_train,y_val = Y.values[tr_index] , Y.values[val_index]
        
    eval_set = [(x_val, y_val)]
    
    model =CatBoostClassifier(**cat_params)
    model.fit(x_train, y_train, eval_set = eval_set, verbose = False)
    
    train_preds = model.predict(x_train)    
    val_preds = model.predict_proba(x_val)
    
    print(log_loss(y_val, val_preds))
    
    if test_preds is None:
        test_preds = model.predict_proba(test_df[cols].values)
    else:
        test_preds += model.predict_proba(test_df[cols].values)

print("-" * 50)
test_preds /= 10


Generating the submission file:

In [ ]:
submission1 = pd.read_csv("../input/tabular-playground-series-jun-2021/sample_submission.csv")
submission1['Class_1']=test_preds[:,0]
submission1['Class_2']=test_preds[:,1]
submission1['Class_3']=test_preds[:,2]
submission1['Class_4']=test_preds[:,3]
submission1['Class_5']=test_preds[:,4]
submission1['Class_6']=test_preds[:,5]
submission1['Class_7']=test_preds[:,6]
submission1['Class_8']=test_preds[:,7]
submission1['Class_9']=test_preds[:,8]
submission1.head()

In [ ]:
submission1.to_csv("sub_1.csv",index=False)

Trying out XGB Classifier:

In [ ]:
from xgboost import XGBClassifier
import xgboost
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_auc_score, make_scorer

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25,random_state=50)

xgb_clf = XGBClassifier(learning_rate = 0.1, n_estimators = 200,use_label_encoder = False, verbose= None, objective = 'multi:softmax', eval_metric = 'mlogloss',eval_set = [X_test, y_test])

xgb_clf.fit(X_train, y_train)

In [ ]:
y_pred_xgb_pr = xgb_clf.predict_proba(X_test)

y_pred_xgb = xgb_clf.predict(X_test)

In [ ]:
acc_scr_xgb = accuracy_score(y_test, y_pred_xgb)
acc_scr_xgb

In [ ]:
auc_score_xgb = roc_auc_score(y_test, y_pred_xgb_pr, multi_class = 'ovr')
auc_score_xgb

In [ ]:
clf_xgb = classification_report(y_test, y_pred_xgb)
print(clf_xgb)

In [ ]:
cfm_xgb = confusion_matrix(y_test, y_pred_xgb)

In [ ]:
plt.figure(figsize = (8, 6))
sns.heatmap(cfm_xgb, annot = True)
plt.xlabel("Predicted")
plt.ylabel("Actual")

In [ ]:
feat_importances = xgb_clf.feature_importances_

In [ ]:
feat_list = X_train.columns

In [ ]:
plt.figure(figsize = (15,15))
plt.barh(list(feat_list), feat_importances)
plt.yticks(list(range(0,77)))
plt.xlabel("Feature score")
plt.ylabel("Feature list")

Model Explainability with SHAP library:

In [ ]:
import shap

In [ ]:
explainer = shap.TreeExplainer(xgb_clf)

shap_values = explainer.shap_values(X_test)

In [ ]:
shap.summary_plot(shap_values, X_test)